# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [240]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [241]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [242]:
%store -r numerosKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[5]
print numerosKalaba
%store numerosKalaba

[5]
Stored 'numerosKalaba' (list)


In [243]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"21-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]

In [244]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [245]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K5/


/Users/gilles/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: 
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K5/
Les attributs de ADJ ne sont pas cohérents
Morphosyntax => Genre, Nombre, Cas
Gloses => Genre, Nombre


NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2
lumière lumière lumière
lumières lumière lumière
head stems,NOM,N3
demande demande demande
demandes demande demande
soeur soeur soeur
soeurs soeur soeur
head stems,NOM,N4
head stems,VER
head stems,VER,VI
head stems,VER,VT
nourrit nourrir nourrir
lever lever lever-NOM
excuse excuser excuse
PB ADJ A1 None
PB ADJ A2 None

=======ERREURS========

Formes homographes "lumière" pour "lumière" et "lumière"
Formes homographes "lumières" pour "lumière" et "lumière"
Formes homographes "demande" pour "demande" et "demande"
Formes homographes "demandes" pour "demande" et "demande"
Formes homographes "soeur" pour "soeur" et "soeur"
Formes homographes "soeurs" pour "soeur" et "soeur"
Formes homographes "nourrit" pour "nourrir" et "nourrir"
Formes homographes "lever" pour "lever" et "lever-NOM"
Formes homographes "excuse" pour "exc

In [246]:
getExemple(stems["PREP"])

u'pr\xe8s'

In [247]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

grand
forêt.N1.A
près


In [248]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=A',
 u'VER, Temps=PRS, Pers=3Sg, Genre=B',
 u'VER, Temps=PRS, Pers=3Sg, Genre=C',
 u'VER, Temps=PRS, Pers=3Sg, Genre=D',
 u'VER, Temps=PRS, Pers=3Pau, Genre=A',
 u'VER, Temps=PRS, Pers=3Pau, Genre=B',
 u'VER, Temps=PRS, Pers=3Pau, Genre=C',
 u'VER, Temps=PRS, Pers=3Pau, Genre=D',
 u'VER, Temps=PRS, Pers=3Pl, Genre=A',
 u'VER, Temps=PRS, Pers=3Pl, Genre=B',
 u'VER, Temps=PRS, Pers=3Pl, Genre=C',
 u'VER, Temps=PRS, Pers=3Pl, Genre=D',
 u'VER, Temps=PST, Pers=3Sg, Genre=A',
 u'VER, Temps=PST, Pers=3Sg, Genre=B',
 u'VER, Temps=PST, Pers=3Sg, Genre=C',
 u'VER, Temps=PST, Pers=3Sg, Genre=D',
 u'VER, Temps=PST, Pers=3Pau, Genre=A',
 u'VER, Temps=PST, Pers=3Pau, Genre=B',
 u'VER, Temps=PST, Pers=3Pau, Genre=C',
 u'VER, Temps=PST, Pers=3Pau, Genre=D',
 u'VER, Temps=PST, Pers=3Pl, Genre=A',
 u'VER, Temps=PST, Pers=3Pl, Genre=B',
 u'VER, Temps=PST, Pers=3Pl, Genre=C',
 u'VER, Temps=PST, Pers=3Pl, Genre=D']

In [249]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [250]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')
regles.getRules("VER","Pers=3Pau")

[]

In [251]:
test=Lexeme("RAD","PRO","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
#test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

del Cas


Nombre,Pau,Pl,Sg
Genre,,,
A,RAD,RAD,RAD
B,RAD,RAD,RAD
C,RAD,RAD,RAD
D,RAD,RAD,RAD


In [252]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'A': {('DoDal', u'p\xe8re'),
  ('Dodor', u'lumi\xe8re'),
  ('DuguN', u'for\xeat'),
  ('Nipet', u'bras'),
  ('dimuN', u'homme'),
  ('dobin', u'fille'),
  ('kides', u'soeur'),
  ('maguS', u'obscurit\xe9'),
  ('makert', u'sorcier'),
  ('mejaN', u'h\xe9ros'),
  ('mikot', u'marais'),
  ('moger', u'tornade'),
  ('mogul', u'histoire'),
  ('mowat', u'milieu'),
  ('pikag', u'chemin'),
  ('pomaD', u'ma\xeetre'),
  ('tarem', u'm\xe8re')},
 u'B': {('Dodor', u'lumi\xe8re'),
  ('Nubif', u'esprit'),
  ('Sebep', u'place'),
  ('Soniv', u'mort'),
  ('Tobaz', u'cr\xe9ature'),
  ('Zenuk', u'caillou'),
  ('bolok', u'lever-nom'),
  ('gubal', u'crapaud'),
  ('jegap', u'jardin'),
  ('jidaf', u'manoir'),
  ('kobud', u'feu'),
  ('nuZun', u'peau'),
  ('oderi', u'Hoderi'),
  ('pabov', u'b\xfbcher'),
  ('ragop', u'nouvelle-n'),
  ('tapiT', u'femme'),
  ('temos', u'vote'),
  ('tiTum', u'sortil\xe8ge'),
  ('tibefu', u'grimoire'),
  ('tinup', u'demande'),
  ('vuTus', u'vengeance')},
 u'C': {('Sures', u'voleuse'),
 

In [253]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

fuir.VT.V1 Zetak
feu.N4.B kobud
pendre.VT.V1 Duwif
Clemencia.N3.C klemansja
remercier.VT.V2 sotuN
retourner.VI.V2 bagut
oeil.N1.C revover
créature.N1.B Tobaz
dernier-n.N2.D ruwoN
quatre manaN
PRO g
nourrir.VT.V1 mutase
Hoderi.N2.B oderi
rejoindre.VT.V2 riluS
placer.VT.V1 makuS
changer.VT.V2 tedut
construction.N2.D gotuz
reconnaître.VT.V1 Dagam
protéger.VT.V1 muTib
courir.VI.V1 tidag
proximité.N3.D serkam
père.N4.A DoDal
vert Tonud
pleurer-act.VI.V2 Sosak
enlever.VT.V1 bonug
disparaître.VT.V1 toZune
surplomber.VT.V1 toNet
bûcher.N3.B pabov
connaître.VT.V1 Degem
Elphaba.N2.D elfaba
tourner.VT.V2 tiTig
demander.VT.V1 furun
nouveau dikeT
demande.N1.D tinup
raison.N4.C favet
milieu.N2.A mowat
livre.N4.D ladop
douleur.N1.C segot
demande.N3.B tinup
embrasser.VT.V1 pafit
héros.N3.A mejaN
nouvelle-n.N3.B ragop
noyer.VT.V2 kuZab
troisième okarina
héroïque bugoN
en oj
fille.N1.A dobin
accueillir.VT.V2 peZip
gargouille.N2.D gevop
légume.N2.C fivaD
autre bunig
DEM b
nuire.VI.V1 gariv
inquiéter.VT.V

In [254]:
ding()